In [1]:
!pip install -qU \
  langchain==0.1.1 \
  langchain-community==0.0.13 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  pinecone-client==3.1.0 \
  pinecone-datasets==0.7.0 \
  pinecone-notebooks==0.1.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.35.18 requires botocore<1.36.0,>=1.35.18, but you have botocore 1.35.16 which is incompatible.


In [2]:
!pip install PyPDF2 pdfplumber
!pip install pdfplumber
!pip install cohere transformers

  Using cached botocore-1.35.18-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.35.18-py3-none-any.whl (12.5 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.16
    Uninstalling botocore-1.35.16:
      Successfully uninstalled botocore-1.35.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.0 requires botocore<1.35.17,>=1.35.16, but you have botocore 1.35.18 which is incompatible.


In [3]:
import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

resume_text = extract_text_from_pdf('/content/Resume-Yash.pdf')
documents = resume_text.split('\n')  # Split the text into smaller chunks (lines or paragraphs)

In [4]:
import os

if not os.environ.get("8195b7e2-f5a8-4f29-84b5-f0d43bdcd888"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [12]:
from pinecone import Pinecone

api_key = os.environ.get("8195b7e2-f5a8-4f29-84b5-f0d43bdcd888")

pc = Pinecone(api_key=api_key)

In [13]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [14]:
index_name = 'resume-retrieval-augmentation-fast'

In [15]:
import time

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

pc.create_index(
        index_name,
        dimension=4096,
        metric='dotproduct',
        spec=spec
    )

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [16]:
index = pc.Index(index_name)
time.sleep(1)
index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [17]:
import cohere
cohere_client = cohere.Client('VoGQ1PX4QgbUh0v6ZfdK9QwNZVAIvBj7WUGHZv7a')
embeddings = cohere_client.embed(texts=documents).embeddings
vectors = [{'id': str(i), 'values': emb} for i, emb in enumerate(embeddings)]
index.upsert(vectors)

{'upserted_count': 56}

In [18]:
import numpy as np

def retrieve_documents(query, top_k=5):
    query_embedding = cohere_client.embed(texts=[query]).embeddings[0]
    for i, val in enumerate(query_embedding):
        if not isinstance(val, float) or not np.isfinite(val):
            print(f"Invalid value at index {i}: {val}")
            query_embedding[i] = 0.0

    query_embedding = [float(np.clip(val, -1, 1)) for val in query_embedding]

    result = index.query(vector=[query_embedding], top_k=top_k)
    return [documents[int(match['id'])] for match in result['matches']]

In [19]:
def generate_answer(query):
    context = retrieve_documents(query)
    combined_context = ' '.join(context)

    response = cohere_client.generate(
        model='command-r-plus-08-2024',
        prompt=f"Context: {combined_context}\n\nQuestion: {query}\n\nAnswer:",
        max_tokens=100
    )
    return response.generations[0].text


In [21]:
queries = [
    "What is the name, email, etc of the candidate in the resume?",
    "Is the candidate suitable for GenAI Internship Job according to the resume?",
    "What skills does the candidate have according to the resume?"
]

for query in queries:
    print(f"Query: {query}")
    print(f"Answer: {generate_answer(query)}\n")


Query: What is the name, email, etc of the candidate in the resume?
Answer: The candidate's name is Yash Shukla, and their email address is shukla02yash@gmail.com. They are a final year B.Tech student in Computer Science and Engineering with a specialization in Artificial Intelligence and Machine Learning at the University. Yash is proficient in computer skills and can be contacted via email or mobile at +91 9871537875.

Query: Is the candidate suitable for GenAI Internship Job according to the resume?
Answer: Yes

Query: What skills does the candidate have according to the resume?
Answer: The candidate's skills according to the resume include programming languages such as Scala, C/C++, Python, MySQL, HTML, CSS, and R. They also possess technical skills in data science, data analysis, data structures, algorithms, and probabilistic data. Additionally, the candidate has experience in web development, robotic process automation, database management, and intelligence research.

